In [3]:
import pandas as pd
import numpy as np
from binance.client import AsyncClient, Client
from tqdm import tqdm_notebook as tqdm
import json
import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_strategies(df, long_window, short_window, is_simple=True):
    sign = None
    strategies = []
    
    if is_simple:
        df['mean_long'] = df['price'].rolling(long_window).mean()
        df['mean_short'] = df['price'].rolling(short_window).mean()
    if not is_simple:
        df['mean_long'] = df['price'].ewm(span = long_window, adjust=False, min_periods=long_window).mean()
        df['mean_short'] = df['price'].ewm(span = short_window, adjust=False, min_periods=short_window).mean()

    for row in df.iterrows():
        mean_short, mean_long = row[1]['mean_short'], row[1]['mean_long']
        
        if pd.isna(mean_long):
            strategies.append(pd.NA)
            continue

        local_sign = (mean_short - mean_long) >= 0
        strategy = 0

        if sign == None:
            sign = local_sign

        if local_sign != sign: # знак поменялся
            if local_sign: # поменялся с - на +
                strategy = 1
            else: # поменялся с + на -
                strategy = -1

        sign = local_sign
        strategies.append(strategy)
        
    return strategies

In [3]:
def backtest(df, long_window, short_window, is_simple=True):
    df['strategy'] = get_strategies(df, long_window, short_window, is_simple=is_simple)
    df = df.iloc[(long_window-1):,]
    df['strategy'] = pd.to_numeric(df['strategy'])
    df['price'] = pd.to_numeric(df['price'])
    df.loc[:, 'buy_price'] = df.loc[:, 'price'] * df.loc[:, 'strategy']
    df = df[df['strategy'] != 0]
    df['next_price'] = df['buy_price'].shift(-1)
    df = df[~df['next_price'].isna()]
    df['profit'] = np.where(
        df['strategy'] == -1,
        (df['price'] - df['next_price']) + df['buy_price'] * COMMISION_RATE - df['next_price'] * COMMISION_RATE,
        -(df['next_price'] + df['price']) - df['buy_price'] * COMMISION_RATE + df['next_price'] * COMMISION_RATE)

    return df['profit'].sum()

In [4]:
def calc_profit(df):
    df = df[~df['strategy'].isna()]
    df['strategy'] = pd.to_numeric(df['strategy'])
    df['price'] = pd.to_numeric(df['price'])
    df.loc[:, 'buy_price'] = df.loc[:, 'price'] * df.loc[:, 'strategy']
    df = df[df['strategy'] != 0]
    df['next_price'] = df['buy_price'].shift(-1)
    df = df[~df['next_price'].isna()]
    df['profit'] = np.where(
        df['strategy'] == -1,
        (df['price'] - df['next_price']) + df['buy_price'] * COMMISION_RATE - df['next_price'] * COMMISION_RATE,
        -(df['next_price'] + df['price']) - df['buy_price'] * COMMISION_RATE + df['next_price'] * COMMISION_RATE)

    return df['profit'].sum()

In [5]:
with open('binance_credentials.json', 'r') as json_file:
    credentials = json.load(json_file)

In [6]:
API_KEY = credentials['API_KEY']
API_SECRET = credentials['API_SECRET']

In [7]:
client = AsyncClient(API_KEY, API_SECRET)

In [8]:
MINUTES_NUMBER = 10_000

In [9]:
COMMISION_RATE = 0.0001

In [10]:
historical_klines = pd.DataFrame()
async for kline in await client.get_historical_klines_generator("BTCUSDT", Client.KLINE_INTERVAL_1MINUTE, f"{MINUTES_NUMBER} minutes ago UTC"):
    close_price = kline[4]
    close_time = kline[6]
    historical_klines = historical_klines.append({'time': pd.to_datetime(close_time, unit='ms'), 'price': close_price}, ignore_index=True)

In [11]:
historical_klines, future_klines = historical_klines[:int(MINUTES_NUMBER/2)], historical_klines[int(MINUTES_NUMBER/2):]

In [ ]:
calc_profit(df)

In [ ]:
df = df[~df['strategy'].isna()]
df['strategy'] = pd.to_numeric(df['strategy'])
df['price'] = pd.to_numeric(df['price'])
df.loc[:, 'buy_price'] = df.loc[:, 'price'] * df.loc[:, 'strategy']
df = df[df['strategy'] != 0]
df['next_price'] = df['buy_price'].shift(-1)
df = df[~df['next_price'].isna()]
df['profit'] = np.where(
    df['strategy'] == -1,
    (df['price'] - df['next_price']) + df['buy_price'] * COMMISION_RATE - df['next_price'] * COMMISION_RATE,
    -(df['next_price'] + df['price']) - df['buy_price'] * COMMISION_RATE + df['next_price'] * COMMISION_RATE)

In [ ]:
df.profit.describe()

In [ ]:
df = historical_klines.copy()

recount_interval = 15
short_window = 15
long_window = 20

for i, row in tqdm(enumerate(future_klines.iterrows()), total=int(MINUTES_NUMBER/2)):
    close_price = row[1]['price']
    close_time = row[1]['time']
    df = df.append({'price': close_price, 'time': close_time}, ignore_index=True)
    
    if i % recount_interval == 0:
        results = {}

        for s in range(5, 45, 5):
            for l in range(s + 5, 50, 5):
                results[backtest(df.copy(), l, s, is_simple=False)] = (l, s)
                
        long_window, short_window = results[max(results)]
        print("Updated strategy: ", long_window, short_window)
    
    df['strategy'] = get_strategies(df.copy(), long_window, short_window, is_simple=False)

In [ ]:
calc_profit(df)

In [ ]:
df = df[~df['strategy'].isna()]
df['strategy'] = pd.to_numeric(df['strategy'])
df['price'] = pd.to_numeric(df['price'])
df.loc[:, 'buy_price'] = df.loc[:, 'price'] * df.loc[:, 'strategy']
df = df[df['strategy'] != 0]
df['next_price'] = df['buy_price'].shift(-1)
df = df[~df['next_price'].isna()]
df['profit'] = np.where(
    df['strategy'] == -1,
    (df['price'] - df['next_price']) + df['buy_price'] * COMMISION_RATE - df['next_price'] * COMMISION_RATE,
    -(df['next_price'] + df['price']) - df['buy_price'] * COMMISION_RATE + df['next_price'] * COMMISION_RATE)

In [ ]:
df.profit.describe()